In [ ]:
class active_set:
    def __init__(self,beta,lambda_):
        self.beta = beta
        self.lambda_ = lambda_
        self.active_set ={}
    def add_to_active_set(self,index):
        if index not in self.active_set:
            self.active_set[index] = self.beta[index]
    def remove_from_active_set(self,index):
        if index in self.active_set:
            del self.active_set[index]
    def get_active_set(self):
        """获取当前的活动集"""
        return self.active_set

    def get_complementary_set(self):
        """获取活动集的补集"""
        full_set_indices = set(range(len(self.beta)))  # 全集的索引
        active_set_indices = set(self.active_set.keys())  # 活动集的索引
        complementary_indices = full_set_indices - active_set_indices  # 计算补集的索引

        # 构建补集
        complementary_set = {index: self.beta[index] for index in complementary_indices}

        return complementary_set

In [18]:
import numpy as np
import math
class Poissonmethod:
    @staticmethod
    def poissonexpression(X,beta):
        return np.exp(np.dot(X,beta))
    @staticmethod
    def pmf(k,mu):
        return (mu**k * math.exp(-mu)) / math.factorial(k)
    @staticmethod
    def loglikelihood(X,beta,y):
        mu=Poissonmethod.poissonexpression(X,beta)
        log_likelihood = np.sum(y * np.log(mu) - mu- np.log(np.array([math.factorial(i) for i in y])))
        return log_likelihood
    @staticmethod
    def MLEwithL1(X,beta,y,L1):
        loglikelihood=Poissonmethod.loglikelihood(X,beta,y)
        return -loglikelihood+L1*np.sum(np.abs(beta))
    def gradient(X,beta,y):
        mu=Poissonmethod.poissonexpression(X,beta)
        gradident=np.dot(X.T,(y-mu))
        return -gradident

    @staticmethod
    def d_beta_d_lambda(X, beta, y, L1):
        sign_beta = np.sign(beta)
        d_beta_d_lambda = np.zeros_like(beta)

        for j in range(len(beta)):
            numerator = sign_beta[j]
            denominator = np.sum(X[:, j] ** 2 * np.exp(X @ beta))
            d_beta_d_lambda[j] = numerator / denominator

        return d_beta_d_lambda
    




In [21]:
import numpy as np
from Poissonmethod import Poissonmethod

class Poisson_regularization_path:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.beta = np.zeros(X.shape[1])  # 初始化beta为0
        self.lambda_=np.inf
        beta_total=np.sum(np.abs(self.beta))
        #lossfunction
        self.FunctionJ=Poissonmethod.MLEwithL1(self.X,self.beta,self.y,self.lambda_)+self.lambda_*beta_total
        self.J_with_Partialdiff=Poissonmethod.gradient(X,self.beta,y)+np.sign(self.beta)
        self.d_lambda_d_beta=Poissonmethod.d_beta_d_lambda(self.X,self.beta,self.y,self.lambda_)
        self.active_set=active_set(self.beta,self.lambda_)
        
        
        

In [25]:
    def analytical_solution(self):
        X = self.X
        y = self.y
        n, p = X.shape
        
        # 初始化L1
        L1 = 0.1
        epsilon = 1e-6  # 添加一个小的偏置量，防止L1变为零
        
        for j in range(p):
            sum_term = np.sum(X[:, j] * (y - np.exp(X @ self.beta)))
            print(f'Iteration {j}, sum_term: {sum_term}')
            self.beta[j] = (1 / (L1)) * sum_term
        
        # 更新L1
        L1 = np.linalg.norm(self.beta, 1) / (np.linalg.norm(X.T @ (y - np.exp(X @ self.beta)), 1) + epsilon)
        print(f'Updated L1: {L1}')
        
        return self.beta, L1

    

In [26]:
     def d_lambda_d_beta(self):
        X = self.X
        beta = self.beta
        n, p = X.shape
        
        d_lambda_d_beta = np.zeros_like(beta)
        
        for j in range(p):
            numerator = np.sum(X[:, j] ** 2 * np.exp(np.clip(X @ beta, -100, 100)))  # Clip to avoid overflow
            denominator = np.sign(beta[j])
            if denominator != 0:
                d_lambda_d_beta[j] = numerator / denominator
            else:
                d_lambda_d_beta[j] = 0
        
        return d_lambda_d_beta

    

In [28]:
    def calculate_cl(self, l):
        n = self.X.shape[0]
        self.cl = 0  # 初始化累加变量

        for i in range(n):  # 遍历每一行
            self.cl += -self.X[i,l] * (-self.y[l] - np.exp(self.X[i, l] * self.beta[l]))

        return self.cl   

In [ ]:
    def calculate_dl(self,l):
        n=self.X.shape[0]
        self.dl = 0
        
        for i in range(n):
            self.dl += self.X[i, l]**2 *np.exp(self.X[i][l]*self.beta[l])*self.d_lambda_d_beta()[l]
        return self.dl

In [ ]:
    def upadate_active_set(self,l):
        active_set=self.active_set.get_active_set(self)
        lambda_=active_set.lambda_
        cl=Poisson_regularization_path.calculate_cl(self,l)
        non_active_set=self.active_set.get_complementary_set(self)
        if l in active_set:
            if abs(cl) < lambda_:
                self.active_set.remove_from_active_set(l)
        if l in non_active_set:
            if abs(cl)==lambda_:
                self.active_set.add_to_active_set(l)

In [ ]:
    def step_length(self,active_set):
        beta=self.beta
        lambda_ = active_set.lambda_
        non_active_set=active_set.get_complementary_set(self)
        cl=Poisson_regularization_path.calculate_cl(self,lambda_)
        dl=Poisson_regularization_path.calculate_dl(self,lambda_)
        d_lambda_add=float('inf')
        for i in  non_active_set:
           d_lambda_add= min((lambda_-cl)/(dl-1),(lambda_+cl)/(-dl-1))
        d_lambda_remove=float('inf')
        for i in active_set:
            d_lambda_remove=min(-beta[i]*Poisson_regularization_path.d_lambda_d_beta(self))
        d_delta=min(d_lambda_remove,d_lambda_add)
        return d_delta
        

In [ ]:
    def predict(self,active_set):
        lambda_=active_set.lambda_
        beta=active_set.beta
        newbeta=np.zeros_like(beta)
        step_length=self.step_length(active_set)
        d_lambda_db=self.step_length()
        newbeta = np.zeros_like(beta)
        for i in range(len(beta)):
            newbeta[i] = beta[i] + step_length* d_lambda_db[i]

        return newbeta
    

In [ ]:
    def predict_and_correct(self):
    # 使用 predict 函数找到一个近似值
        beta_approx = self.predict(self.active_set)
    
    # 使用坐标下降法修正 beta 值
        beta_corrected = self.coordinate_descent(beta_approx)
    
    # 更新 beta
        self.beta = beta_corrected

In [ ]:
    def update_beta(self, beta_l, gradient):
        """根据梯度更新beta[l]"""
        # 使用梯度下降法更新 beta_l
        beta_l_updated = beta_l - self.learning_rate * gradient
        return beta_l_updated

In [ ]:
    def coordinate_descent(self, beta_approx):
        max_iterations = 100  # 设置最大迭代次数
        tolerance = 1e-6  # 设置收敛容忍度
        beta = beta_approx.copy()
    
    
        for iteration in range(max_iterations):
            beta_old = beta.copy()
            for l in self.active_set.get_active_set():
            # 计算梯度更新
                gradient = self.calculate_gradient(l)
                beta[l] = self.update_beta(beta[l], gradient)  # 这里需要实现 update_beta 函数
            
        # 检查收敛
            if np.linalg.norm(beta - beta_old) < tolerance:
                break
    
        return beta

In [ ]:
    def regularization_path(self):
    # 初始化
        self.initialize()
    
        while True:
        # 计算步长并更新 lambda
            d_delta = self.step_length()
            self.lambda_ -= d_delta
        
        # 更新活动集
            self.update_active_set()
        
        # 预测并修正 beta
            self.predict_and_correct()
        
        # 终止条件：活动集不再变化，或 lambda 变得非常小
            if self.lambda_ < 1e-6 or not self.active_set.get_active_set():
                break
    
        return self.beta

In [6]:

    X = np.array([[1, 2, 3], 
              [4, 5, 6], 
              [7, 8, 9]])

    y = np.array([1, 2, 3])  # 简单的目标变量# 初始化 Poisson_regularization_path 实例


# 测试正则化路径函数
    beta_result = regularization_path()

# 输出结果print("最终的 beta 值:", beta_result)
    print(beta_result)

NameError: name 'regularization_path' is not defined